# April 2022 Questions

## 04/13 Question

Suppose we are given an array of n integers which represent the value of some stock over time. Assuming you are allowed to buy the stock exactly once and sell the stock once, what is the maximum profit you can make? Can you write an algorithm that takes in an array of values and returns the maximum profit?
    


    
    
 For example, if you are given the following array:


    
    
[2, 7, 1, 8, 2, 8, 14, 25, 14, 0, 4, 5]
    


    

    
 The maximum profit you can make is 24 because you would buy the stock when its price is 1 and sell when it's 25. Note that we cannot make 25, because the stock is priced at 0 after it is priced at 25 (e.g you can't sell before you buy).
 
 ## Approach
 
 To solve this problem, I will first initiate a max_profit variable to track the highest profit calculated thus far.
 
I will then loop through each number in the provided array. For each of these loops, I will compare the first number to all those afterward and take the difference between them. If the difference, which would be a profit from buying/selling a stock, is higher than the current max profit value then it will be set as the new max profit value.
 
At the end of the function, I will return the max profit variable.

In [1]:
def sell_stock(arr):
    max_profit=0
    for i in range(len(arr)-1):
        current_price=arr[i]
        for future_price in arr[i+1:]:
            if future_price-current_price>max_profit:
                max_profit = future_price-current_price
    return max_profit

## 04/11 Question

Suppose your team interviews undergraduate candidates across many different colleges. You are looking to check which candidates scored the highest from each college.

Given the below tables, write a SQL query (using a window function) to show which candidates scored the highest from each college.

Table: candidateColleges


|college_id	|candidate_name|
|---|---|
|123456	|john_smith|
|123456	|sarah_daniels|
|123457	|tim_cook|
|123457	|lisa_perelli|
|123457	|jenny_west|
|123457	|karl_tran|
|123457	|tammy_turner|

    

    
Table: candidateInterviews

Each row represents a unique interview/candidate as well as the candidate's score (an integer from 1-5, with 5 representing the highest possible score).

|interview_id|	candidate_name|	interview_score|
|---|---|---|
|12|	john_smith|	4|
|22|	sarah_daniels|	3|
|23|	tim_cook|	3|
|25|	lisa_perelli|	5|
|26|	jenny_west|	2|
|27|	karl_tran|	2|
|28|	tammy_turner|	4|


### Approach

To perform the above query, I will:
- join the two tables on candidate_name
- use the rank function to create a new column, ranking scores by college_id using the partition method/syntax
- display the candidate name, college, score and rank ordered by college/score for better college-level comparison

### Solution

SELECT a.candidate_name AS Name,  <br>
       b.college_id as College,<br>
       a.interview_score AS Score, <br>
       RANK () OVER ( PARTITION BY b.college_id ORDER BY a.interview_score DESC) AS Score_Rank_By_College <br>
FROM  candidateInterviews a JOIN  candidateColleges b on a.candidate_name=b.candidate_name <br>
order by College desc, Score Desc

## 04/04 Question

Given an array of integers, print the array in alternating min/max order. The first element should be the first maximum, second element should be the first minimum, third element should be second maximum, etc.

For example:
    
#Given the following array:
  
arr[] = [10, 2, 11, 3, 7, 4, 1] 

#Your function should return:

11, 1, 10, 2, 7, 3, 4

### Approach

To solve this proble, I will create a function that will do as follows:
- Take in an array
- Initiate an empty res array to store the final result
- Initiate a queue with our input array
- Use a while loop to go through the sorted data
    - While there are more than 1 items in the queue, pop and popleft and append the results to the res variable
- When there is 0 or 1 item in the array, append the remaining value (if there is one) to the res variable
- return res variable

### Solution

In [46]:
from collections import deque

def min_max_order(arr):
    res=[]
    queue=deque(sorted(arr))
    while len(queue)>1:
        res.append(queue.pop())
        res.append(queue.popleft())
    if queue:
        res.append(queue.pop())
    return res

In [47]:
min_max_order([10,2,11,3,7,4,1,5])

[11, 1, 10, 2, 7, 3, 5, 4]

## 04/01 Question

Given the following table containing customer transaction data:


      
Table name: orders 


      
|order_id|	channel|	date|	month|	revenue|
|---|---|---|---|---|
|0	|1	|Online	|2018-09-01	|09	|100|
|1	|2	|Online	|2018-09-03	|09	|125|
|2	|3	|In_store	|2018-10-11	|10	|200|
|3	|4	|In_store	|2018-08-21	|08	|80|
|4	|5	|Online	|2018-08-13	|08	|200|
|5	|6	|Online	|2018-10-29	|10	|100|
    
Write SQL queries to answer the following:
  
- Return all rows in the data table with transaction values over \$101.
- Return all rows in the data table with transaction values over \$101 that were purchased through the Online channel.
- Return all rows in the data table with transaction values that were either over \$101 OR were purchased In_store.
- Return the minimum transaction amount for both channels as well as the corresponding order id.

### Solution 

SELECT * FROM orders WHERE revenue>101 ;

SELECT * FROM orders WHERE revenue>101 AND channel = "Online";

SELECT * FROM orders WHERE revenue>101 OR channel = "In_store";

SELECT channel, order_id, min(revenue) FROM orders GROUP BY channel;

